The way I have coded the model makes it easy to play around with. Just toggle with the model parameters.

Loading refined data and reshaping it for the model

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Load preprocessed data
X_train = np.load("../refined_data/X_train.npy")
y_train = np.load("../refined_data/y_train.npy")
X_val = np.load("../refined_data/X_val.npy")
y_val = np.load("../refined_data/y_val.npy")
X_test = np.load("../refined_data/X_test.npy")
y_test = np.load("../refined_data/y_test.npy")


IMG_SIZE = (48, 48)
NUM_CLASSES = 7
X_train = tf.repeat(X_train, 3, axis=-1)
X_val = tf.repeat(X_val, 3, axis=-1)
X_test = tf.repeat(X_test, 3, axis=-1)
y_train = to_categorical(np.argmax(y_train, axis=-1) , num_classes=NUM_CLASSES)
y_val = to_categorical(np.argmax(y_val, axis=-1), num_classes=NUM_CLASSES)
y_test = to_categorical(np.argmax(y_test, axis=-1), num_classes=NUM_CLASSES)

print("Data loaded successfully!")
print(f"Training data shape: {X_train.shape}, Labels shape: {y_train.shape}")
print(f"Validation data shape: {X_val.shape}, Labels shape: {y_val.shape}")
print(f"Test data shape: {X_test.shape}, Labels shape: {y_test.shape}")

Data loaded successfully!
Training data shape: (22968, 48, 48, 3), Labels shape: (22968, 7)
Validation data shape: (5741, 48, 48, 3), Labels shape: (5741, 7)
Test data shape: (28709, 48, 48, 3), Labels shape: (28709, 7)


Model Parameters (Edit here)

In [2]:
DROPOUT_RATE = 0.3  # Dropout rate for regularization
LEARNING_RATE = 1e-4  # Learning rate

DENSE_UNITS = [128, 64]  # 2 Dense layer units, add/remove as needed
 
epochs = 10
batch_size = 128

Model Built (Don't edit it is basic structure)

In [3]:
# Load Pretrained VGG16 Model (without the top layer)
import requests

url = "https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
response = requests.get(url, verify=False)  # Bypass SSL verification (not recommended for production)
with open("vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", "wb") as f:
    f.write(response.content)

# Now use the local file
vgg_expert = VGG16(weights="vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

model = Sequential()
model.add(vgg_expert)  # Add VGG16 layers as feature extractor

model.add(GlobalAveragePooling2D())  # Pooling layer to reduce dimensions
for units in DENSE_UNITS:
    model.add(Dense(units, activation='relu'))  # Dense layers
    model.add(Dropout(DROPOUT_RATE))  # Dropout for regularization

model.add(Dense(NUM_CLASSES, activation='softmax'))  # Output layer 

# Compile the model
model.compile(
    loss='categorical_crossentropy',  # For multi-class classification
    optimizer=Adam(learning_rate=LEARNING_RATE), 
    metrics=['accuracy']  # Track accuracy during training
)

# Display the model summary to inspect the architecture
model.summary()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,789,063 (56.42 MB)

 Trainable params: 14,789,063 (56.42 MB)

 Non-trainable params: 0 (0.00 B)

Train Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

Accuracy

In [5]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")

225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step
Test Accuracy: 24.71%


Save Trained Model

In [ ]:
import joblib 

joblib.dump(model, 'emotion_model.pkl')
print("Model saved successfully!")